### 📸✨ **What Is Speckle Noise And Why Does It Happen?**  

When we form a **SAR image**, our goal is to **coherently combine the echoes from multiple spatial positions** to achieve a **high-resolution image**.  
But this process of coherent combination has a downside: **Speckle noise**. 🌪️  

---

### 🔍 **Where Does Speckle Noise Come From?**

Consider a relatively **homogeneous surface** — like grass 🌾 — that’s **rough on the scale of the radar wavelength**.

In SAR imaging, the backscatter from a **single pixel** over such a surface is the **sum of echoes from many small scatterers** within that pixel area.  

Each scatterer reflects part of the radar signal, and these reflections combine depending on their **individual phases and amplitudes**. ↖️➕↙️➕↗️➕↘️  

- 📡 The combined echoes can **interfere constructively** (adding up) or **destructively** (canceling out).  
- 🎲 The result is a **seemingly random pattern** of bright and dark spots — what we call **speckle noise**.  
- 📉 Smaller pixels contain **fewer scatterers**, which can lead to **higher variability** in the summed signal.

> ⚠️ **Note:**  
Calling it *speckle noise* is actually a bit misleading — it’s not just random noise, but the result of **real, physical interference** between **true echoes from the target surface**! 💡

---

### 📉 **What Does Speckle Look Like?**  
- Instead of a smooth image, **speckle** appears as a **granular or salt-and-pepper-like texture** over homogeneous areas. 🌌  
- It’s especially noticeable over surfaces that should have **uniform backscatter**, like fields, asphalt, or wavy water. 🌊🏞️  

---

### ❌ **Why Is Speckle A Problem?**  
- It **reduces image quality** by introducing high-frequency fluctuations that are **unrelated to actual physical features**.  
- It **complicates image interpretation**, especially when trying to estimate properties of **homogeneous surfaces**.  

---

### 📖 **The Need for Multilooking**  

**Multilooking** is a powerful technique used in SAR processing to **reduce speckle** and improve the **visual quality and interpretability** of images. But how does it actually work? 🤔  

---

### 🔄 **How Multilooking Works**  

The basic idea is to **divide the synthetic aperture into several parts**, called **looks**. 📏✂️  

- 🧩 **Creating Looks:**  
  - We split the **virtual aperture** into multiple **sub-apertures** (or sub-arrays).  
  - In **spotlight mode**, each sub-aperture can generate an image of the **same ground area**, because the radar beam is **continuously illuminating** the target throughout the collection. 🔁  
  - Even though each look uses a **shorter synthetic aperture**, it still captures the **same scene** from slightly different perspectives. 👀  

---

### 📉 **The Trade-Off**  

- 📏 **Worse Cross-Range Resolution:**  
  - Each individual look has **lower cross-range resolution**, since resolution in this dimension is **proportional to aperture length**.  
  - By dividing the aperture, we reduce its length, which means **blurrier images per look**. 😕  

---

### 🎯 **Why It Works**  

- 🎲 **Speckle is random** — it varies independently in each look because the sub-apertures are **non-overlapping**.  
- 📊 When we **average the look images together** (in **magnitude** or **power**, not phase), the **random fluctuations caused by speckle** tend to **cancel out**. ⬆️⬇️ 
- ✅ The result is a **cleaner image** with **reduced speckle**, leading to **easier interpretation**.  

---

### ⚖️ **The Trade-Off Recap**  

- ✅ **Multilooking Improves:**  
  - Image readability and usability for interpretation 🗺️  

- ❌ **Multilooking Degrades:**  
  - **Spatial resolution** in the **cross-range direction** 📉  

---

Multilooking gives you a **smoother**, cleaner image — at the cost of some **sharpness**. 

Usually a fair trade, especially when **visual interpretation** or classification is the goal! 🖼️👌


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Simulation parameters
satellite_velocity = 7.5  # km/s (just for visualization, not accurate scale)
collection_time = 10  # seconds of data acquisition
num_frames = 100
altitude = 500  # km (altitude of the satellite)
incidence_angle = 30  # degrees
ground_point = (0.0, 0.0, 0.0)  # Target on the ground

# Calculate aperture length
aperture_length = satellite_velocity * collection_time  # km
x_start, x_end = -aperture_length / 2, aperture_length / 2  # Flight path

# Satellite positions
x_positions = np.linspace(x_start, x_end, num_frames)
y_pos = altitude * np.tan(np.radians(incidence_angle))
y_positions = np.full_like(x_positions, y_pos)
z_positions = np.full_like(x_positions, altitude)

# Define colors for the different looks (sub-arrays)
colors = ['red', 'green', 'blue', 'orange']

# Number of looks (sub-arrays)
num_looks = 4
elements_per_look = num_frames // num_looks

# Create figure
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

# Plot the ground point
ax.scatter(*ground_point, color='k', marker='o', s=50, label='Ground Point')

# Plot radar platform trajectory divided into looks
for i in range(num_looks):
    start_idx = i * elements_per_look
    end_idx = (i + 1) * elements_per_look if i < num_looks - 1 else num_frames
    ax.plot(x_positions[start_idx:end_idx], y_positions[start_idx:end_idx], z_positions[start_idx:end_idx], 
            color=colors[i], lw=2, label=f'Look {i+1}')

# Setting labels and title
ax.set_xlim(x_start - 10, x_end + 10)
ax.set_ylim(-10, y_pos + 10)
ax.set_zlim(0, altitude + 10)
ax.set_title("Dividing the Synthetic Aperture Into Independent Sub-Apertures Called Looks")
ax.set_xlabel("X Position (km)")
ax.set_ylabel("Y Position (km)")
ax.set_zlabel("Altitude (km)")
ax.legend(loc='upper left')

plt.show()


### 🎮 **Exploring the Concept of Multilooking**

Now it’s your turn to **experiment with multilooking!** 🧩✨  

In our **single-look SAR data**, we usually start with **higher resolution in cross-range than in ground range**. This means that after multilooking, we can still achieve **balanced ground range and cross-range resolution** in the final image. ⚖️ 

#### 🛠️ How It Works:
- Suppose we want a final resolution of **1 meter**, but we also want to **average 5 independent looks** to reduce speckle.
- To do that, our **single-look cross-range resolution** must be:  $\frac{1}{5} = 0.2\ \text{meters}$
- The idea is to **divide the synthetic aperture into 5 smaller sub-apertures**, generate one image from each (a **look**), and **average them**.  
- This reduces speckle while maintaining a desirable final resolution. 🎯

---

### 🧪 Try It Yourself!

Use the interactive code below to **adjust the number of looks** and observe how it impacts:
- **Speckle noise**
- **Image clarity**
- **Spatial resolution**

You can also change the level of speckle in the **simulated data** by adjusting the `noise_std` parameter. 🎛️🎲  
See how more noise or fewer looks affects the **visual quality** of the output image! 📉🖼️📈


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Simulation parameters
number_of_looks = 4     # Number of independent looks
noise_std = 0.5         # Adjust speckle level as desired
range_resolution = 1.0  # slant range resolution (m)
cross_range_resolution = range_resolution / number_of_looks  # single-look cross-range resolution

# Radar & Simulation Parameters
c = 299792458            # Speed of light, m/s
wavelength = 0.03        # m
B = c / (2 * range_resolution)  # Chirp bandwidth, Hz
sampling_rate = 4 * B           # Hz
pulse_duration = 10e-6          # s
pulse_width = pulse_duration * c  # m

y_radar = 100.0   # Radar y coordinate (m)
z_radar = 100.0   # Radar z coordinate (m)
radar_r0 = np.sqrt(y_radar**2 + z_radar**2)

range_window = 100  
image_x_size = 50

# Define targets (x, y, z), all with unit amplitude
targets = 2 * np.array(
    [[-5.0, 0.0, 0.0],
     [-4.0, 0.0, 0.0],
     [-3.0, 0.0, 0.0],
     [-2.0, 0.0, 0.0],
     [-1.0, 0.0, 0.0],
     [ 1.0, 0.0, 0.0],
     [ 2.0, 0.0, 0.0],
     [ 3.0, 0.0, 0.0],
     [ 4.0, 0.0, 0.0],
     [ 5.0, 0.0, 0.0],
     [0.0, -5.0, 0.0],
     [0.0, -4.0, 0.0],
     [0.0, -3.0, 0.0],
     [0.0, -2.0, 0.0],
     [0.0, -1.0, 0.0],
     [0.0,  1.0, 0.0],
     [0.0,  2.0, 0.0],
     [0.0,  3.0, 0.0],
     [0.0,  4.0, 0.0],
     [0.0,  5.0, 0.0]]
)

# Derive aperture length and sampling
aperture_angle = wavelength / (2 * cross_range_resolution)
aperture_length = 2 * radar_r0 * np.atan(aperture_angle / 2) 
x_start, x_end = -aperture_length / 2, aperture_length / 2  
delta_x = wavelength * radar_r0 / (2 * image_x_size)
number_of_pulses = int(round(aperture_length / delta_x))

# Radar positions along x, at y_radar, z_radar
x_radar = np.linspace(x_start, x_end, number_of_pulses)
xyz_radar = np.column_stack((
    x_radar,
    np.full(number_of_pulses, y_radar),
    np.full(number_of_pulses, z_radar),
))

# Range axis
radar_r1 = np.linalg.norm(xyz_radar[0, :])
range0 = radar_r0 - range_window/2           
total_time = 2.0 * range_window / c  
num_samples = int(sampling_rate * total_time)
range_axis = np.linspace(range0, range0 + (c * total_time / 2), num_samples, endpoint=False)

def sinc_pulse(r_array, center, width):
    """
    Generate a normalized sinc pulse in terms of range.
    'r_array' : the range axis (similar to time but scaled by c/2).
    'center'  : the peak's range
    'width'   : approximate main-lobe width in range
    """
    y = np.sinc((r_array - center) / width)
    peak = np.max(np.abs(y))
    if peak > 0:
        y /= peak
    return y

# Simulate range-compressed data
range_compressed_data = np.zeros((number_of_pulses, num_samples), dtype=np.complex64)
for i in range(number_of_pulses):
    echo_sum = np.zeros(num_samples, dtype=np.complex64)
    radar_x = x_radar[i]  # Current radar X

    for (tx, ty, tz) in targets:
        # All targets have unit amplitude
        amplitude = 1.0

        # Range from radar to target
        r = np.sqrt((radar_x - tx)**2 + (y_radar - ty)**2 + (z_radar - tz)**2)

        # Two-way phase term => exp(j * 2*pi * (2*r / wavelength))
        # factor of 2*r for round trip
        target_phasor = np.exp(1j * 2 * np.pi * (2 * r / wavelength))

        # Range-domain sinc centered at r, with 'range_resolution' as width
        echo_pulse = sinc_pulse(range_axis, r, range_resolution)
        echo_sum += amplitude * echo_pulse * target_phasor

    range_compressed_data[i, :] = echo_sum

# Simulate Gaussian noise (I + jQ). This mimics spaeckle in the data, producing a return with uniformly distributed phase
np.random.seed(0)
I_noise = np.random.normal(0, noise_std, range_compressed_data.shape)
Q_noise = np.random.normal(0, noise_std, range_compressed_data.shape)
gaussian_noise = I_noise + 1j * Q_noise

# Add noise to the signal
noisy_data = range_compressed_data + gaussian_noise

# Plot the noisy magnitude data
data_to_plot = np.abs(noisy_data)
fig, ax = plt.subplots(figsize=(10, 6))
im = ax.imshow(
    data_to_plot,
    extent=[range_axis[0], range_axis[-1], 0, number_of_pulses],
    aspect='auto',
    origin='lower',
    cmap='jet'
)
ax.set_xlabel("Range (m)")
ax.set_ylabel("Pulse Number")
ax.set_title("Radar Echo Simulation with Simulated Speckle (Magnitude)")
cbar = plt.colorbar(im, ax=ax)
cbar.set_label("Amplitude (Magnitude)")
plt.tight_layout()
plt.show()

In [ ]:
def sar_beamforming(range_vector, range_compressed_data_array, xyz_radar, wavelength, 
                         image_x_extent, image_y_extent, nb_pixels_x, nb_pixels_y):
    """
    Form a 2D image by time-domain delay and sum beamforming (also called back-projection)
    """
    # Number of pulses and range bins in data
    number_of_pulses = range_compressed_data_array.shape[0]
    nb_range_samples = range_compressed_data_array.shape[1]

    # Range sampling
    range_spacing = range_vector[1] - range_vector[0]  # Assume uniform spacing

    # Create output image
    image = np.zeros((nb_pixels_y, nb_pixels_x), dtype=np.complex128)

    # Define x and y pixel coordinates
    x_grid = np.linspace(image_x_extent[0], image_x_extent[-1], nb_pixels_x)
    y_grid = np.linspace(image_y_extent[0], image_y_extent[-1], nb_pixels_y)

    # Loop over each pixel
    for iy, y_val in enumerate(y_grid):
        for ix, x_val in enumerate(x_grid):
            # Loop through the virtual antenna elements (pulses)
            for pulse in range(number_of_pulses):
                # Range from this radar position to the pixel
                pixel_range = np.sqrt((x_val - xyz_radar[pulse,0])**2 + 
                                      (y_val - xyz_radar[pulse,1])**2 + 
                                      (xyz_radar[pulse,2])**2)
                ## Find corresponding sample index (nearest neighbor)
                sample_index = int(round((pixel_range-range_vector[0])/range_spacing))
                if 0 <= sample_index < nb_range_samples:
                    # Calculate the phase we expected an echo from this pixel to have
                    pixel_phase = 2 * np.pi * (2*pixel_range) / wavelength
                    # Use a phasor with opposite phase to align the echoes from all virtual elements
                    pixel_phasor_correction = np.exp(-1j*pixel_phase)
                    # Sum the phase aligned echoes togehter
                    image[iy, ix] += range_compressed_data_array[pulse, sample_index] * pixel_phasor_correction

    image /= number_of_pulses # Normalize image amplitude

    return image, x_grid, y_grid

### First we make a single-look image 👀

In [ ]:
# Define image extent
x_extent = (-15, 15)
y_extent = (-15, 15)
# Define number of pixels. 
nb_pixels_x = 2*int(round(np.abs(x_extent[-1] -  x_extent[0]) / cross_range_resolution))
nb_pixels_y = 2*int(round(np.abs(y_extent[-1] - y_extent[0]) / range_resolution))
print("Number of single-look image pixels (x,y)", nb_pixels_x, nb_pixels_y)

# Perform the back-projection
im, x_grid, y_grid = sar_beamforming(
    range_axis,
    noisy_data,
    xyz_radar,
    wavelength,
    x_extent,
    y_extent,
    nb_pixels_x,
    nb_pixels_y
)

# Plot the SAR image
plt.figure(figsize=(8, 6))

max_val = np.max(10*np.log10(np.abs(im)))
min_val = max_val-20

# Plot the SAR image
plt.imshow(
    10*np.log10(np.abs(im)),
    extent=[x_grid[0], x_grid[-1], y_grid[0], y_grid[-1]],
    origin='lower',
    aspect='auto',
    cmap='jet',
    vmin=min_val,
    vmax=max_val,
)

plt.colorbar(label="Amplitude (Magnitude)")
plt.xlabel("x (m)")
plt.ylabel("y (m)")
plt.title("Single-Look Image (Magnitude)")
plt.tight_layout()
plt.show()

### 🧪 Now Let's Try Multilooking! 👀➕👀➕👀 ➡️ 📉✨  

We’ll divide the **synthetic aperture** into **equal-sized segments** — each one giving us a slightly different **“look”** at the same scene. 👁️👁️👁️  

For each segment, we form an image, and then **average the looks together** to reduce **speckle noise**. 🎲📉

In [ ]:
# Number of independent segments to divide the data
nb_pulses_per_look = int(np.floor(number_of_pulses / number_of_looks))
nb_pixels_x_ml = nb_pixels_x // number_of_looks
print("Number of multilooked image pixels (x,y)", nb_pixels_x_ml, nb_pixels_y)

multilooked_image = np.zeros((nb_pixels_y, nb_pixels_x_ml), dtype=np.complex128)
for i_look in range(number_of_looks):
    # Calculate which pulses from the data to use for this look
    pulse_start = i_look*nb_pulses_per_look
    pulse_end = pulse_start + nb_pulses_per_look
    # Pick the data (i.e. the virtual elements) that correspond to this look
    look_data = noisy_data[pulse_start:pulse_end,:]
    look_radar_pos = xyz_radar[pulse_start:pulse_end:,:]
    look_image, _, _ = sar_beamforming(range_axis, look_data, look_radar_pos, wavelength, 
                                       x_extent, y_extent, nb_pixels_x_ml, nb_pixels_y)
    # Accumulate the squared magnitude (power) of the image
    multilooked_image += np.abs(look_image)**2

# Plot the SAR image
plt.figure(figsize=(8, 6))

ml_image_magnitude = np.sqrt(multilooked_image / number_of_looks)
max_val = np.max(10*np.log10(np.abs(ml_image_magnitude)))
min_val = max_val-20

# Plot the SAR image
plt.imshow(
    10*np.log10(np.abs(ml_image_magnitude)),
    extent=[x_grid[0], x_grid[-1], y_grid[0], y_grid[-1]],
    origin='lower',
    aspect='auto',
    cmap='jet',
    vmin=min_val,
    vmax=max_val,    
)

plt.colorbar(label="Amplitude (Magnitude)")
plt.xlabel("x (m)")
plt.ylabel("y (m)")
plt.title("Multilooked Image (Magnitude)")
plt.tight_layout()
plt.show()

> 📝 **Final Note:**  
The multilooked image we produce is what’s known as a **Ground Range Detected (GRD) image product**.  

🤔 *But what does “detected” actually mean?*  

Well — it just means **amplitude**. “Detection” is what incredibly smart radar engineers call the act of taking the **absolute value of a complex number**. 🧮

Yes, we know — the **terminology in the radar world can be mind-blowingly confusing**. 🙃  

I’ll have a few words (and maybe a small rant) in the final lesson. Stay tuned! 🎤😄
